# Mitigating Disparities in Ranking from Binary Data
_**An example based on the Law School Admissions Council's National Longitudinal Bar Passage Study**_


## Contents

1. [Introduction](#Introduction)
1. [Data](#Data)
1. [Unmitigated Predictor](#Unmitigated-Predictor)
1. [Mitigating Demographic Disparity with Grid Search](#Mitigating-Demographic-Disparity-with-Grid-Search)
   1. [Comparing Probabilistic Predictors using the Dashboard](#Comparing-Probabilistic-Predictors-using-the-Dashboard)
1. [Obtaining Low-Disparity Classifiers](#Obtaining-Low-Disparity-Classifiers)
   1. [Postprocessing](#Postprocessing)
   1. [Exponentiated Gradient](#Exponentiated-Gradient)
   1. [Comparing Classifiers using the Dashboard](#Comparing-Classifiers-using-the-Dashboard)

   
## Introduction

We consider the task of ranking students for admission to law school. We use the data collected in  [Law School Admissions Council's (LSAC) National Longitudinal Bar Passage Study](https://eric.ed.gov/?id=ED469370); specifically, the version downloaded from [Project SEAPHE](http://www.seaphe.org/databases.php). We highlight some of the fairness considerations that come up not only in school admissions, but also in other ranking scenarios. Necessarily, our example is simplified and ignores many real-world considerations specific to school admissions.

The data set contains information about law students collected by LSAC between 1991 and 1997. Some of the information is available at the admission time (such as the undergraduate GPA and LSAT score), and some describes the performance of the students once admitted. We also have access to their self-identified race. Throughout this exercise, we will limit the attention to those self-identified as **black** and **white** (two largest groups).

To help with ranking law school applicants, we will train a model that uses the information that is available about a student at the admission time to predict the probability that they will pass their bar exam. The predictions of our model are intended to be used (among other factors) by admission officers to select the applicants. After training the initial model, we examine differences in the predictions it induces across two groups. We then mitigate these differences using `GridSearch` approach in `fairlearn`.

## Data

We download the data using the `tempeh` package, which already filters the set of students to black and white and splits them into training and test subsets. The training and test data sets are loaded in three parts:

* **X_train**, **X_test**: features describing the training and test data; `tempeh` provides two features: `ugpa` (undegraduate GPA) and `lsat` (LSAT score)

* **y_train**, **y_test**: labels of the training and test data; the labels are 0 or 1, indicating whether a student passed the bar exam by the 2nd attempt

* **A_train**, **A_test**: self-identified race of each student (black or white)

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

# Load the data using the tempeh package
from tempeh.configurations import datasets
dataset = datasets['lawschool_passbar']()

X_train, X_test = dataset.get_X(format=pd.DataFrame)
y_train, y_test = dataset.get_y(format=pd.Series)
A_train, A_test = dataset.get_sensitive_features(name='race', format=pd.Series)

# Combine all training data into a single data frame and glance at a few rows
all_train = pd.concat([X_train, y_train, A_train], axis=1)
display(all_train)

Now, let us examine the data more closely. We look at the distributions of `lsat` and `ugpa` by race (summarized via quartiles), and compare them with the bar passage rates.

In [ ]:
all_train_grouped = all_train.groupby('race')

counts_by_race = all_train_grouped[['lsat']].count().rename(
    columns={'lsat': 'count'})

quartiles_by_race = all_train_grouped[['lsat','ugpa']].quantile([.25, .50, .75]).rename(
    index={0.25: "25%", 0.5: "50%", 0.75: "75%"}, level=1).unstack()

rates_by_race = all_train_grouped[['pass_bar']].mean().rename(
    columns={'pass_bar': 'pass_bar_rate'})

summary_by_race = pd.concat([counts_by_race, quartiles_by_race, rates_by_race], axis=1)
display(summary_by_race)

The vast majority of the students in the study are white. There is a notable gap between white and black students in their incoming academic credentials: the 75th percentile of the LSAT scores of black students is lower than the 25th percentile of the LSAT scores among white students. There is a less severe, but still substantial gap in UGPA. The achievement gap is greatly diminished in terms of the bar passage rate (78% for black students and 97% for white students). The authors of the [LSAC study](https://eric.ed.gov/?id=ED469370) conclude that this justifies admission practices that look beyond LSAT and UGPA. Since we do not have access to such additional variables, in the remaider of this notebook we will seek to build a bar passage predictor from these two variables alone.

## Unmitigated Predictor

We first train a standard logistic regression predictor that does not seek to incorporate any notion of fairness.

In [ ]:
from sklearn.linear_model import LogisticRegression

unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)
unmitigated_predictor.fit(X_train, y_train)

We view the probabilistic predictions produced by the logistic model as scores and evaluate their accuracy in separating positive examples from the negative ones (i.e., the students who pass the bar from those who do not). Because of a high imbalance between positive and negative examples, our accuracy metric is the [area under the ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) (AUC). It is equal to the probability that a randomly chosen positive example is scored above a randomly chosen negative example. An AUC of 0.5 means that the evaluated scores are no better than a random coin flip, whereas AUC of 1.0 means that the evaluated scores are perfect. The AUC metric has two desirable properties: (1) it is not sensitive to the imbalance between positives and negatives, and (2) it is preserved by monotone transformations of the score.

To obtain the AUC values for the overall student population as well as black and white subpopulations, we use the **group metric** variant of the `sklearn` metric [`roc_auc_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html).

In [ ]:
from fairlearn.metrics import group_roc_auc_score

# a convenience function that transforms the result of a group metric call into a data frame
def group_metric_as_df(name, group_metric_result):
    a = pd.Series(group_metric_result.by_group)
    a['overall'] = group_metric_result.overall
    return pd.DataFrame({name: a})

scores_unmitigated = pd.Series(unmitigated_predictor.predict_proba(X_test)[:,1], name="score_unmitigated")
auc_unmitigated = group_metric_as_df("auc_unmitigated",
                                     group_roc_auc_score(y_test, scores_unmitigated, A_test))
display(auc_unmitigated)

We next examine how the unmitigated predictor affects applicants of different race when it is used to score them. We plot the CDFs of the scores it generates for each group. We then consider all possible thresholds on the value of the score, and for each threshold check the fraction of black vs white students above the threshold. The largest observed difference across all possible thresholds is referred to as the **demographic disparity** (see [Agarwal et al. 2018](http://proceedings.mlr.press/v97/agarwal19d.html)). Pictorially, this corresponds to the largest vertical difference between the two CDFs. Note that this disparity metric is preserved under monotone transformations of the scores.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import cumfreq

def compare_cdfs(data, A, num_bins=100):
    cdfs = {}
    assert len(np.unique(A)) == 2
    
    limits = ( min(data), max(data) )
    s = 0.5 * (limits[1] - limits[0]) / (num_bins - 1)
    limits = ( limits[0]-s, limits[1] + s)
    
    for a in np.unique(A):
        subset = data[A==a]
        
        cdfs[a] = cumfreq(subset, numbins=num_bins, defaultreallimits=limits)
        
    lower_limits = [v.lowerlimit for _, v in cdfs.items()]
    bin_sizes = [v.binsize for _,v in cdfs.items()]
    actual_num_bins = [v.cumcount.size for _,v in cdfs.items()]
    
    assert len(np.unique(lower_limits)) == 1
    assert len(np.unique(bin_sizes)) == 1
    assert np.all([num_bins==v.cumcount.size for _,v in cdfs.items()])
    
    xs = lower_limits[0] + np.linspace(0, bin_sizes[0]*num_bins, num_bins)
    
    disparities = np.zeros(num_bins)
    for i in range(num_bins):
        cdf_values = np.clip([v.cumcount[i]/len(data[A==k]) for k,v in cdfs.items()],0,1)
        disparities[i] = max(cdf_values)-min(cdf_values)  
    
    return xs, cdfs, disparities
    
    
def plot_and_compare_cdfs(data, A, num_bins=100, loc='best'):
    xs, cdfs, disparities = compare_cdfs(data, A, num_bins)
    
    for k, v in cdfs.items():
        plt.plot(xs, v.cumcount/len(data[A==k]), label=k)
    
    assert disparities.argmax().size == 1
    d_idx = disparities.argmax()
    
    xs_line = [xs[d_idx],xs[d_idx]]
    counts = [v.cumcount[d_idx]/len(data[A==k]) for k, v in cdfs.items()]
    ys_line = [min(counts), max(counts)]
    
    plt.plot(xs_line, ys_line, 'o--')
    disparity_label = "max disparity = {0:.3f}\nat {1:0.3f}".format(disparities[d_idx], xs[d_idx])
    plt.text(xs[d_idx], 1, disparity_label, ha="right", va="top")
    
    plt.xlabel(data.name)
    plt.ylabel("cumulative frequency")
    plt.legend(loc=loc)
    plt.show()

plot_and_compare_cdfs(scores_unmitigated, A_test)

We see that the largest disparity of about 0.6 occurs at the threshold value 0.94: only 23% of black students, but 83% of white students are above this threshold.

## Mitigating Demographic Disparity with Grid Search

We next show how to mitigate the demographic disparity using the `GridSearch` algorithm of `fairlearn`. We will use this algorithm to obtain several models that achieve various trade-offs between accuracy (measured by AUC) and demographic disparity.

The `GridSearch` variant that we will use was developed for *classification* under demographic parity, but the experiments of
[Agarwal et al. (2018)](http://proceedings.mlr.press/v97/agarwal19d.html) show that it also performs well in optimizing the demographic parity of the continuous-valued probabilistic predictions of logistic models. Unlike standard logistic models, the models obtained by this approach might not be well calibrated, so we use Platt's scaling for [calibration](https://scikit-learn.org/stable/modules/calibration.html). Note that Platt's scaling is a monotone transformation, and so it has no effect on the AUC values or the demographic disparity of the resulting model. However, it makes the predicted scores interpretable as probabilities.

`GridSearch` generates models corresponding to various Lagrange multiplier vectors of the underlying constraint optimization problem. We will compute 41 models on a grid of Lagrange multiplier vectors whose L1-norm is bounded by 10. For details on how the search works, refer to Section 3.4 of [Agarwal et. al (2018)](http://proceedings.mlr.press/v80/agarwal18a.html). The following cell may take a couple of minutes to run:

In [ ]:
from fairlearn.reductions import GridSearch, DemographicParity
from sklearn.calibration import CalibratedClassifierCV

sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid_size=41,
                   grid_limit=10)

sweep.fit(X_train, y_train, sensitive_features=A_train)

calibrated_predictors = []
for result in sweep.all_results:
    calibrated = CalibratedClassifierCV(base_estimator=result.predictor, cv='prefit', method='sigmoid')
    calibrated.fit(X_train, y_train)
    calibrated_predictors.append(calibrated)

We next assess the accuracy and disparity of the obtained predictors in a scatter plot, with *x* axis showing the worst-case AUC among the two subpopulations and *y* axis showing the demographic disparity. Ideal models would be in the bottom right.

In [ ]:
def auc_disparity_sweep_plot(predictors, names, marker='o', scale_size=1, zorder=-1):
    roc_auc = np.zeros(len(predictors))
    disparity = np.zeros(len(predictors))
    
    for i in range(len(predictors)):
        preds = predictors[i].predict_proba(X_test)[:,1]
        roc_auc[i] = group_roc_auc_score(y_test, preds, A_test).minimum
        _, _, dis = compare_cdfs(preds, A_test)
        disparity[i] = dis.max()
        
    plt.scatter(roc_auc, disparity,
                s=scale_size * plt.rcParams['lines.markersize'] ** 2, marker=marker, zorder=zorder)
    for i in range(len(roc_auc)):
        plt.annotate(names[i], (roc_auc[i], disparity[i]), xytext=(3,2), textcoords="offset points", zorder=zorder+1)
    plt.xlabel("worst-case AUC")
    plt.ylabel("demographic disparity")
    
auc_disparity_sweep_plot(calibrated_predictors, names=range(len(calibrated_predictors)))
auc_disparity_sweep_plot([unmitigated_predictor], names=[''], marker='*', zorder=1, scale_size=5)
plt.show()

The model 33 has the lowest disparity, but its worst-case AUC is essentially the same as that of a coin flip. The unmitigated model, marked as a star, has a good worst-case AUC, but large disparity. We examine models 35 and 36: their AUC values are well above 0.6 and they substantially reduce the demographic disparity compared with the unmitigated model:

In [ ]:
scores_model35 = pd.Series(calibrated_predictors[35].predict_proba(X_test)[:,1], name="score_model35")
scores_model36 = pd.Series(calibrated_predictors[36].predict_proba(X_test)[:,1], name="score_model36")

auc_model35 = group_metric_as_df("auc_model35",
                                 group_roc_auc_score(y_test, scores_model35, A_test))
auc_model36 = group_metric_as_df("auc_model36",
                                 group_roc_auc_score(y_test, scores_model36, A_test))
display(pd.concat([auc_model35, auc_model36, auc_unmitigated], axis=1))
plot_and_compare_cdfs(scores_model35, A_test)
plot_and_compare_cdfs(scores_model36, A_test)
plot_and_compare_cdfs(scores_unmitigated, A_test)

### Comparing Probabilistic Predictors using the Dashboard

Next, we compare the three predictors above (unmitigated, model 35 and model 36) using `FairlearnDashboard`. The dashboard currently does not evaluate the demographic disparity of probabilistic scores, but instead evaluates the disparity in mean predictions&mdash;in this case, this amounts to the difference between mean predictions for the white and black subpopulations.

In [ ]:
from fairlearn.widget import FairlearnDashboard
FairlearnDashboard(sensitive_features=A_test, sensitive_feature_names=['Race'],
                   y_true=y_test,
                   y_pred={"unmitigated": scores_unmitigated, "model35": scores_model35, "model36": scores_model36})

## Obtaining Low-Disparity Classifiers

### Postprocessing

In [ ]:
from fairlearn.postprocessing import ThresholdOptimizer

class LogisticRegressionAsRegression:
    def __init__(self, logistic_regression_estimator):
        self.logistic_regression_estimator = logistic_regression_estimator
    
    def fit(self, X, y):
        self.logistic_regression_estimator.fit(X, y)
    
    def predict(self, X):
        # use predict_proba to get real values instead of 0/1, select only prob for 1
        scores = self.logistic_regression_estimator.predict_proba(X)[:,1]
        return scores


balanced_index_pass0 = y_train[y_train==0].index 
balanced_index_pass1 = y_train[y_train==1].sample(n=balanced_index_pass0.size, random_state=0).index
balanced_index = balanced_index_pass0.union(balanced_index_pass1)

pp_estimator = ThresholdOptimizer(
    unconstrained_predictor=LogisticRegressionAsRegression(unmitigated_predictor),
    constraints="demographic_parity")

pp_estimator.fit(X_train.iloc[balanced_index,:], y_train.iloc[balanced_index],
                 sensitive_features=A_train.iloc[balanced_index])

In [ ]:
from fairlearn.metrics import group_mean_prediction

scores_pp = pd.Series(pp_estimator.predict(X_test, sensitive_features=A_test), name="scores_post")
auc_pp = group_metric_as_df("auc_post",
                            group_roc_auc_score(y_test, scores_pp, A_test))
selection_pp = group_metric_as_df("selection_post",
                                  group_mean_prediction(y_test, scores_pp, A_test))
display(pd.concat([auc_pp, selection_pp], axis=1))

### Exponentiated Gradient

In [ ]:
from fairlearn.reductions import ExponentiatedGradient

XA_train = pd.concat([X_train, A_train=='black'], axis=1)
XA_test = pd.concat([X_test, A_test=='black'], axis=1)

expgrad_X = ExponentiatedGradient(LogisticRegression(solver='liblinear', fit_intercept=True),
                                  constraints=DemographicParity(),
                                  eps=0.01)
expgrad_XA = ExponentiatedGradient(LogisticRegression(solver='liblinear', fit_intercept=True),
                                   constraints=DemographicParity(),
                                   eps=0.01)


expgrad_X.fit(X_train.iloc[balanced_index,:], y_train.iloc[balanced_index],
              sensitive_features=A_train.iloc[balanced_index])
expgrad_XA.fit(XA_train.iloc[balanced_index,:], y_train.iloc[balanced_index],
              sensitive_features=A_train.iloc[balanced_index])

In [ ]:
scores_expgrad_X = pd.Series(expgrad_X.predict(X_test), name="scores_expgrad_X")
scores_expgrad_XA = pd.Series(expgrad_XA.predict(XA_test), name="scores_expgrad_XA")

auc_expgrad_X = group_metric_as_df("auc_expgrad_X",
                            group_roc_auc_score(y_test, scores_expgrad_X, A_test))
selection_expgrad_X = group_metric_as_df("selection_expgrad_X",
                                         group_mean_prediction(y_test, scores_expgrad_X, A_test))
auc_expgrad_XA = group_metric_as_df("auc_expgrad_XA",
                            group_roc_auc_score(y_test, scores_expgrad_XA, A_test))
selection_expgrad_XA = group_metric_as_df("selection_expgrad_XA",
                                         group_mean_prediction(y_test, scores_expgrad_XA, A_test))

display(pd.concat([auc_pp, selection_pp, auc_expgrad_X, selection_expgrad_X, auc_expgrad_XA, selection_expgrad_XA], axis=1))

### Comparing Classifiers using the Dashboard

In [ ]:
FairlearnDashboard(sensitive_features=A_test, sensitive_feature_names=['Race'],
                   y_true=y_test,
                   y_pred={"postprocessing": scores_pp, "expgrad_X": scores_expgrad_X, "expgrad_XA": scores_expgrad_XA})